In [ ]:
# !pip install fastapi uvicorn nest-asyncio


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile, HTTPException
from PIL import Image
import io
import tensorflow as tf
import numpy as np

# Permitir que FastAPI se ejecute en Jupyter
nest_asyncio.apply()

# Inicializar la aplicación de FastAPI
app = FastAPI()

# Cargar el modelo entrenado de Keras en formato HDF5 (.h5)
model = tf.keras.models.load_model('../Results/ensemble_model.h5')  # Asegúrate de que la ruta sea correcta

# Clases de tu modelo (asegúrate de que el orden es correcto)
class_names = ['Chinche salivosa', 'Clororis', 'Hoja sana', 'Roya naranja', 'Roya purpura']

@app.post("/predict/")
async def predict_image(file: UploadFile = File(...)):
    try:
        # Leer la imagen subida
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))

        # Convertir la imagen a RGB y redimensionarla
        image = image.convert("RGB")
        image = image.resize((256, 256))  # Asegúrate de que coincide con INPUT_SHAPE

        # Convertir la imagen a un array de NumPy y a float32
        img_array = np.array(image).astype(np.float32)

        # Añadir dimensión para el batch
        img_array = np.expand_dims(img_array, axis=0)

        # Realizar la predicción
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        class_name = class_names[predicted_class]
        confidence = float(np.max(predictions))  # Asegúrate de convertir a float

        # Imprimir para depuración
        print("Predicciones:", predictions)
        print("Índice de clase predicha:", predicted_class)
        print("Clase predicha:", class_name)
        print("Confianza:", confidence)

        return {"prediction": class_name, "confidence": confidence}
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error al procesar la imagen: {str(e)}")

# Iniciar el servidor de FastAPI
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8001)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at C:\Users\ealda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\uvicorn\server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\ealda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\uvicorn\main.py", line 577, in run
    server.run()
  File "C:\Users\ealda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\uvicorn\server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ealda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nest_asyncio.py"

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicciones: [[1. 0. 0. 0. 0.]]
Índice de clase predicha: 0
Clase predicha: Chinche salivosa
Confianza: 1.0
INFO:     127.0.0.1:56922 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Predicciones: [[0. 0. 0. 0. 1.]]
Índice de clase predicha: 4
Clase predicha: Roya purpura
Confianza: 1.0
INFO:     127.0.0.1:56935 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
Predicciones: [[0. 0. 0. 0. 1.]]
Índice de clase predicha: 4
Clase predicha: Roya purpura
Confianza: 1.0
INFO:     127.0.0.1:56942 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
Predicciones: [[1. 0. 0. 0. 0.]]
Índice de clase predicha: 0
Clase predicha: Chinche salivosa
Confianza: 1.0
INFO:     127.0.0.1:56953 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Predicciones: [[1.6981954e-26 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00]]
Índice de clase predicha: 4
Clase predicha: Roya

In [31]:
import requests

# URL de la API (usa la URL pública generada por ngrok)
url = "https://0088-34-142-189-227.ngrok-free.app/predict/"

# Subir una imagen para predecir (ajusta la ruta de la imagen según tu ubicación)
with open("../Chinche.jpg", "rb") as image_file:
    response = requests.post(url, files={"file": image_file})

# Verificar si la respuesta es correcta antes de decodificar
if response.status_code == 200:
    try:
        json_response = response.json()  # Intentar decodificar JSON
        print(json_response)
    except ValueError as e:
        print("Error al decodificar JSON:", e)
        print("Contenido de la respuesta:", response.text)
else:
    print(f"Error en la solicitud: {response.status_code}")
    print("Contenido de la respuesta:", response.text)


{'prediction': 'Roya purpura', 'confidence': 1.0}
